In [2]:
import random
from PIL import Image
import numpy as np
import pandas as pd
import shutil
import glob
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

### Gensim pretrain MODEL

In [3]:
import gensim.downloader
from gensim.models import KeyedVectors

In [4]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-200')
# glove-wiki-gigaword-200   #'glove-twitter-200'
# glove-wiki-gigaword-100   #glove-twitter-100'
# word2vec-google-news-300  #fasttext-wiki-news-subwords-300  #'glove-wiki-gigaword-300'
# glove-wiki-gigaword-50    #glove-twitter-50

In [5]:
out_dim=len(glove_vectors[0])  #WordVector的Output_dimension

## Setting

In [6]:
dataset = 'AWA2'
########################################
train_path = Path("./data/{}/IMG_backoff/train".format(dataset))
test_path = Path("./data/{}/IMG_backoff/test".format(dataset))
val_path = Path("./data/{}/IMG_backoff/val".format(dataset))

'''儲存path在Train_Data處理時候才定義。 格式: './{}_{}c/data_{}/'.format(dataset, len(train_classes),out_dim)'''
# if dataset == 'AWA2':

"儲存path在Train_Data處理時候才定義。 格式: './{}_{}c/data_{}/'.format(dataset, len(train_classes),out_dim)"

## Transform_define:

In [7]:
def WordVectortransform(load_path):
    classes = []
    images = []
    img_vecs = []
    class_labels = []
    label_c = 0
    for class_name in load_path.iterdir():
        c = class_name.name.rpartition(".")[-1]
        # print(c)
        if c in glove_vectors:
            class_images = list(class_name.iterdir())
            classes.append(c)
            images += class_images
            img_vecs += [glove_vectors[c]]*len(class_images)
            for k in class_name.iterdir():
                class_labels.append(label_c)
            label_c += 1
    return classes, images, img_vecs, class_labels

### Train Data

In [11]:
train_classes = []
train_images = []
train_img_vecs = []
train_class_labels = []

train_classes, train_images, train_img_vecs, train_class_labels = WordVectortransform(
    train_path)
path = './{}_{}c/'.format(dataset, len(train_classes))
if not os.path.isdir(path):
    os.mkdir(path)
path = './{}_{}c/data_{}/'.format(dataset, len(train_classes),out_dim)
if not os.path.isdir(path):
    os.mkdir(path)

#### Check

In [12]:
print(len(train_classes))
print(len(train_images))
print(len(train_img_vecs))
print(len(train_class_labels))
print(len(train_class_labels))

33
21057
21057
21057
21057


In [13]:
train_img_vecs = np.stack(train_img_vecs)
print(train_img_vecs.shape)

(21057, 200)


In [14]:
# 資料打亂打平均用的，但這樣label可能會亂掉，Training可以用。
grouped = list(zip(train_images, train_img_vecs, train_class_labels))
random.shuffle(grouped)
train_images, train_img_vecs, train_class_labels = zip(*grouped)

### Val_Data

In [15]:
val_classes = []
val_images = []
val_img_vecs = []
val_class_labels = []


val_classes, val_images, val_img_vecs, val_class_labels = WordVectortransform(
    val_path)

#### Check

In [16]:
print(len(val_classes))
print(len(val_images))
print(len(val_img_vecs))
print(val_classes)
print(len(val_class_labels))

33
5268
5268
['bat', 'wolf', 'skunk', 'walrus', 'sheep', 'rhinoceros', 'mole', 'mouse', 'rabbit', 'squirrel', 'dolphin', 'fox', 'tiger', 'deer', 'cow', 'ox', 'weasel', 'collie', 'dalmatian', 'buffalo', 'giraffe', 'moose', 'elephant', 'gorilla', 'otter', 'antelope', 'beaver', 'zebra', 'hamster', 'lion', 'horse', 'bobcat', 'chihuahua']
5268


In [17]:
val_img_vecs = np.stack(val_img_vecs)
print(val_img_vecs.shape)

(5268, 200)


In [18]:
grouped_val = list(zip(val_images, val_img_vecs, val_class_labels))
random.shuffle(grouped_val)
val_images, val_img_vecs, val_class_labels = zip(*grouped_val)

### Test Data

In [19]:
test_classes = []
test_images = []
test_img_vecs = []
test_class_labels = []

test_classes, test_images, test_img_vecs, test_class_labels = WordVectortransform(
    test_path)

#### Check

In [20]:
print(len(test_classes))
print(len(test_images))
print(len(test_img_vecs))
print(test_classes)
print(len(test_class_labels))

7
4654
4654
['hippopotamus', 'pig', 'leopard', 'seal', 'chimpanzee', 'rat', 'raccoon']
4654


In [21]:
test_img_vecs = np.stack(test_img_vecs)
print(test_img_vecs.shape)

(4654, 200)


### Data_Save

In [22]:
def Save(Type, images, img_vecs, classes, class_labels):
    np.save(path+'{}_img_name.npy'.format(Type), images)
    np.save(path+'{}_img_vec.npy'.format(Type), img_vecs)
    np.save(path+'{}_class.npy'.format(Type), classes)
    np.save(path+'{}_img_label.npy'.format(Type), class_labels)

In [23]:
Save('train', train_images, train_img_vecs, train_classes, train_class_labels)

Save('val', val_images, val_img_vecs, val_classes, val_class_labels)

Save('test', test_images, test_img_vecs, test_classes, test_class_labels)

In [24]:
'''
np.save(path+'train_img_name.npy', train_images)
np.save(path+'train_img_vec.npy', train_img_vecs)
np.save(path+'train_class.npy', train_classes)
np.save(path+'train_img_label.npy', train_class_labels)

np.save(path+'val_img_name.npy', val_images)
np.save(path+'val_img_vec.npy', val_img_vecs)
np.save(path+'val_img_class.npy', val_classes)
np.save(path+'val_img_label.npy', val_class_labels)

np.save(path+'test_img_name.npy', test_images)
np.save(path+'test_img_vec.npy', test_img_vecs)
np.save(path+'test_img_class.npy', test_classes)
np.save(path+'test_img_label.npy', test_class_labels)
'''

"\nnp.save(path+'train_img_name.npy', train_images)\nnp.save(path+'train_img_vec.npy', train_img_vecs)\nnp.save(path+'train_class.npy', train_classes)\nnp.save(path+'train_img_label.npy', train_class_labels)\n\nnp.save(path+'val_img_name.npy', val_images)\nnp.save(path+'val_img_vec.npy', val_img_vecs)\nnp.save(path+'val_img_class.npy', val_classes)\nnp.save(path+'val_img_label.npy', val_class_labels)\n\nnp.save(path+'test_img_name.npy', test_images)\nnp.save(path+'test_img_vec.npy', test_img_vecs)\nnp.save(path+'test_img_class.npy', test_classes)\nnp.save(path+'test_img_label.npy', test_class_labels)\n"

In [25]:
del glove_vectors